**Общая информация**

**Срок сдачи:** 29 марта 2021, 08:30 

**Штраф за опоздание:** по 1 баллу за 24 часа задержки. Через 5 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0220, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
2. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_openml, fetch_20newsgroups

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KDTree
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from scipy.spatial import distance_matrix
from scipy.spatial.distance import cdist
import plotly.express as px
from tqdm import tqdm

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. 
Для подсчета расстояний можно использовать функции [отсюда](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

In [4]:
class MyKNeighborsClassifier(BaseEstimator):
    
    def __init__(self, n_neighbors, algorithm='brute', metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm
        self.metric = metric
    
    def fit(self, X, y):
        self.x = np.array(X)
        self.y = np.array(y)
    
    def predict(self, X):
        X = np.array(X)
        if (self.algorithm == 'brute'):
            result = np.empty([0])
            dist = cdist(self.x, X, metric = self.metric)
            args = np.argsort(dist, axis=0, kind='quicksort', order=None)
            result = (self.y[args])[:self.n_neighbors]
            return result[0]
        else:
            tree = KDTree(self.x, metric = self.metric)              
            dist, ind = tree.query(X, k=self.n_neighbors)
            ind = (ind[:,:1])
            result = (self.y[ind]).flatten()
            return result
    

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [5]:
iris = datasets.load_iris()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [7]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [8]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [9]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. 

In [10]:
%time clf.fit(X_train, y_train)

Wall time: 0 ns


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [11]:
%time my_clf.fit(X_train, y_train)

Wall time: 0 ns


In [12]:
%time clf.predict(X_test)

Wall time: 2.99 ms


array([0, 2, 0, 0, 1, 0, 1, 1, 2, 0, 1, 2, 1, 2, 2])

In [13]:
%time my_clf.predict(X_test)

Wall time: 0 ns


array([0, 2, 0, 0, 1, 0, 1, 1, 2, 0, 1, 2, 1, 2, 2])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. Точность не должна уступать значению KNN из sklearn. 

In [21]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [22]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [23]:
%time clf.fit(X_train, y_train)

Wall time: 1e+03 µs


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [24]:
%time my_clf.fit(X_train, y_train)

Wall time: 0 ns


In [25]:
%time clf.predict(X_test)

Wall time: 2.99 ms


array([2, 0, 1, 1, 1, 0, 1, 2, 2, 2, 1, 0, 0, 0, 2])

In [26]:
%time my_clf.predict(X_test)

Wall time: 994 µs


array([2, 0, 1, 1, 1, 0, 1, 2, 2, 2, 1, 0, 0, 0, 2])

In [27]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [28]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [29]:
data = newsgroups['data']
target = newsgroups['target']

Преобразуйте текстовые данные из data с помощью [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Словарь можно ограничить по частотности.

In [30]:
vectorizer = CountVectorizer(max_df = 0.01, min_df = 0.005)
X = vectorizer.fit_transform(data)
X_vec = X.toarray()

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [31]:
from sklearn.model_selection import KFold, cross_val_score
kf = KFold(n_splits=3, shuffle =True)

In [32]:
classifiers = []
for train, test in kf.split(X_vec, target):
    clf = MyKNeighborsClassifier(n_neighbors = 2, algorithm = 'brute')
    X_train, X_test = X_vec[train], X_vec[test]
    Y_train, Y_test = target[train], target[test]
    clf.fit(X_train, y_train)
    classifiers.append(clf)

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальную точность в среднем на валидации на 3 фолдах).
Постройте график зависимости средней точности от количества соседей. Можно рассмотреть число соседей от 1 до 10.

In [33]:
def optimal_neighbours(X, y, algorithm = 'brute', metric = 'euclidean'):
        array = np.arange(1, 11)
        accuracy_array = np.zeros(10)
        i = 0
        for k in tqdm(array):
            clf = MyKNeighborsClassifier(n_neighbors = k, algorithm = algorithm, metric = metric)
            accuracy_array[i] = cross_val_score(clf, X, y, cv = 3, scoring = 'accuracy').mean()
            i += 1
        return accuracy_array, np.argmax(accuracy_array) + 1

In [ ]:
euclid_accuracy, euclid_neighbors = optimal_neighbours(X_vec, target)

In [ ]:
df = pd.DataFrame(np.array(euclid_accuracy), np.arange(1,11))
fig = px.line(df, title='Nighbours accuracy')
fig.show()

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf векторизацию( sklearn.feature_extraction.text.TfidfVectorizer)

Сравните модели, выберите лучшую.

In [ ]:
cosine_accuracy, cosine_neighbors = optimal_neighbours(X_vec, target, metric = 'cosine')

In [34]:
tfidfVect = TfidfVectorizer(max_df = 0.01, min_df = 0.005)
X = tfidfVect.fit_transform(data)
X_vec = X.toarray()

In [ ]:
euclid_accuracy, euclid_neighbors = optimal_neighbours(X_vec, target)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим точность вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.

In [ ]:
data = newsgroups['data']
target = newsgroups['target']
X_vec2 = tfidfVect.transform(test_data).toarray()
clf = MyKNeighborsClassifier(n_neighbors = 10, metric = 'cosine')
clf.fit(X_vec, target)
y_pred = clf.predict(X_vec2)
accuracy_score(y_pred, target)